referensi https://www.analyticsvidhya.com/blog/2021/04/forward-feature-selection-and-its-implementation/

In [1]:
# #instal library mlxtend
# %pip install mlxtend

In [2]:
# Import Library
import pandas as pd
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time

In [3]:
#import dataset
df = pd.read_csv('/home/jupyter-17523142/Rama/Dataset/dataset_rama.csv')
X = df.drop(['No', 'Mango Cultivars', 'Vit C (mg/100g)', 'TA (mg/100g)', 'SSC (oBrix)', 'label'], axis = 1)
y_vitc = df.loc[:, 'Vit C (mg/100g)']

# separate dataset into train and test
x_train, x_test, y_train, y_test = train_test_split(
    X, y_vitc, test_size = 0.3, random_state=0)

## Seleksi Fitur Forward Selection dengan Linear Regression

In [4]:
# calling the Linear Regression model
lr_model =  LinearRegression()
sfs = sfs(lr_model, k_features=100, forward=True, verbose=2, scoring='neg_root_mean_squared_error', n_jobs=-1)

In [5]:
%%time 

sfs = sfs.fit(x_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 205 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 488 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done 853 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done 1298 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done 1557 out of 1557 | elapsed:   10.5s finished

[2022-07-13 06:48:03] Features: 1/100 -- score: -13.787313513773904[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 252 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 818 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 1556 out of 1556 | elapsed:    3.0s finished

[2022-07-13 06:48:06] Features: 2/100 -- score: -12.378759342766646[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent

CPU times: user 19min 58s, sys: 39.5 s, total: 20min 37s
Wall time: 20min 14s


[Parallel(n_jobs=-1)]: Done 1299 out of 1458 | elapsed:   19.9s remaining:    2.4s
[Parallel(n_jobs=-1)]: Done 1458 out of 1458 | elapsed:   20.0s finished

[2022-07-13 07:08:07] Features: 100/100 -- score: -14649175192.940521

In [6]:
feat_names = list(sfs.k_feature_names_)
print("Fitur yang diambil adalah: ")
print(feat_names)
len(feat_names)

Fitur yang diambil adalah: 
['1009.6', '1020.8', '1023.2', '1040.8', '1041.7', '1043.4', '1045.9', '1047.1', '1047.6', '1048.4', '1050.5', '1051', '1052.2', '1052.7', '1054', '1057.4', '1060', '1062.6', '1065.7', '1070.5', '1081.7', '1084.8', '1085.3', '1085.7', '1086.6', '1087.1', '1087.6', '1088.9', '1089.4', '1089.8', '1091.2', '1094.4', '1094.9', '1096.3', '1098.6', '1100.5', '1101.4', '1104.2', '1114.2', '1115.2', '1143.2', '1149.3', '1158', '1191.5', '1200.3', '1299', '1323.5', '1382.1', '1390.2', '1391', '1397.7', '1406', '1409.1', '1411.4', '1414.5', '1422.2', '1424.6', '1425.4', '1427.7', '1436.4', '1438', '1439.6', '1446', '1446.8', '1450.9', '1452.5', '1454.1', '1464', '1466.5', '1485', '1485.8', '1490.1', '1490.9', '1494.4', '1518.9', '1521.6', '1527.8', '1528.7', '1536', '1536.9', '1540.5', '1555.3', '1559.1', '1560', '1563.8', '1654.6', '1659.9', '1660.9', '1664.1', '1666.3', '1669.5', '1674.9', '1680.3', '1681.4', '1732', '1741.3', '1763.8', '1900.8', '2331.6', '2367.8']

100

# Prediksi Linear Regression

In [7]:
#cross validation 10-fold
cv = KFold(n_splits=10, random_state=1, shuffle=True)

In [8]:
%%time

#tentukan metode scoring yang digunakan
metrics = {'rmse': 'neg_root_mean_squared_error',
               'mse': 'neg_mean_squared_error',
               'mae': 'neg_mean_absolute_error',
               'r2': 'r2'} 

n_feat = [100]

for nfeat in n_feat:
        print("==================================================")
        
        #ambil n fitur input hasil seleksi fitur SFS
        x_train_selected = x_train[feat_names].iloc[:,0:nfeat]
        x_test_selected = x_test[feat_names].iloc[:,0:nfeat]

        #Create a Linear Regression
        lr_model = LinearRegression()
        
        #Train the model using the training sets
        lr_model.fit(x_train_selected, y_train)
        y_pred_lr = lr_model.predict(x_test_selected)

        #hitung score model dari data train
        scores = cross_validate(lr_model, x_train_selected, y_train, scoring=metrics, cv=cv, return_train_score=True)

        print("MSE model Linear Regression data Train dengan " + str(nfeat) + " fitur: " 
              + str(abs(round(scores['train_mse'].mean(), 2))))
        print("RMSE model Linear Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs(round(scores['train_rmse'].mean(), 2))))
        print("MAE model Linear Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs(round(scores['train_mae'].mean(), 2))))
        print("R2 model Linear Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs((round(scores['train_r2'].mean(), 2)))))    
        print("----------------------------")        
        print("MSE model Linear Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_lr), 2))))
        print("RMSE model Linear Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_lr, squared = False), 2))))
        print("MAE model Linear Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_absolute_error(y_test, y_pred_lr), 2))))
        print("R2 model Linear Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(r2_score(y_test, y_pred_lr), 2))))
        print(" ")
        print("==================================================")

MSE model Linear Regression data Train dengan 100 fitur: 23.35
RMSE model Linear Regression data Train dengan 100 fitur:4.82
MAE model Linear Regression data Train dengan 100 fitur:2.41
R2 model Linear Regression data Train dengan 100 fitur:0.88
----------------------------
MSE model Linear Regression data Test dengan 100 fitur:1.9287666027630716e+16
RMSE model Linear Regression data Test dengan 100 fitur:138880041.86
MAE model Linear Regression data Test dengan 100 fitur:68424627.7
R2 model Linear Regression data Test dengan 100 fitur:111833965884860.45
 
CPU times: user 6.32 s, sys: 259 ms, total: 6.57 s
Wall time: 279 ms


# Prediksi Random Forest Regressor

## 100 Trees

In [9]:
%%time

#tentukan metode scoring yang digunakan
metrics = {'rmse': 'neg_root_mean_squared_error',
               'mse': 'neg_mean_squared_error',
               'mae': 'neg_mean_absolute_error',
               'r2': 'r2'} 

n_feat = [100]
n_trees = [100]

for nfeat in n_feat:
    for ntrees in n_trees: 
        print("==================================================")
        
        #ambil n fitur input hasil seleksi fitur SFS
        x_train_selected = x_train[feat_names].iloc[:,0:nfeat]
        x_test_selected = x_test[feat_names].iloc[:,0:nfeat]

        #Create a Random Forest Regression
        rfg_model = RandomForestRegressor(n_estimators=ntrees, random_state=100)
        
        #Train the model using the training sets
        rfg_model.fit(x_train_selected, y_train)
        y_pred_rfg = rfg_model.predict(x_test_selected)

        #hitung score model dari data train
        scores = cross_validate(rfg_model, x_train_selected, y_train, scoring=metrics, cv=cv, return_train_score=True)

        print("MSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur: " 
              + str(abs(round(scores['train_mse'].mean(), 2))))
        print("RMSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs(round(scores['train_rmse'].mean(), 2))))
        print("MAE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs(round(scores['train_mae'].mean(), 2))))
        print("R2 model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs((round(scores['train_r2'].mean(), 2)))))    
        print("----------------------------")
        print("MSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_rfg), 2))))
        print("RMSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))))
        print("MAE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))))
        print("R2 model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(r2_score(y_test, y_pred_rfg), 2))))
        print(" ")
        print("==================================================")

MSE model Linear Regression data Train dengan 100 fitur: 39.01
RMSE model Linear Regression data Train dengan 100 fitur:6.24
MAE model Linear Regression data Train dengan 100 fitur:4.77
R2 model Linear Regression data Train dengan 100 fitur:0.8
----------------------------
MSE model Linear Regression data Test dengan 100 fitur:176.79
RMSE model Linear Regression data Test dengan 100 fitur:13.3
MAE model Linear Regression data Test dengan 100 fitur:10.56
R2 model Linear Regression data Test dengan 100 fitur:0.03
 
CPU times: user 7.23 s, sys: 78.6 ms, total: 7.31 s
Wall time: 3.51 s


## 150 Trees

In [10]:
%%time

#tentukan metode scoring yang digunakan
metrics = {'rmse': 'neg_root_mean_squared_error',
               'mse': 'neg_mean_squared_error',
               'mae': 'neg_mean_absolute_error',
               'r2': 'r2'} 

n_feat = [100]
n_trees = [150]

for nfeat in n_feat:
    for ntrees in n_trees: 
        print("==================================================")
        
        #ambil n fitur input hasil seleksi fitur SFS
        x_train_selected = x_train[feat_names].iloc[:,0:nfeat]
        x_test_selected = x_test[feat_names].iloc[:,0:nfeat]

        #Create a Random Forest Regression
        rfg_model = RandomForestRegressor(n_estimators=ntrees, random_state=100)
        
        #Train the model using the training sets
        rfg_model.fit(x_train_selected, y_train)
        y_pred_rfg = rfg_model.predict(x_test_selected)

        #hitung score model dari data train
        scores = cross_validate(rfg_model, x_train_selected, y_train, scoring=metrics, cv=cv, return_train_score=True)

        print("MSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur: " 
              + str(abs(round(scores['train_mse'].mean(), 2))))
        print("RMSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs(round(scores['train_rmse'].mean(), 2))))
        print("MAE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs(round(scores['train_mae'].mean(), 2))))
        print("R2 model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs((round(scores['train_r2'].mean(), 2)))))    
        print("----------------------------")
        print("MSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_rfg), 2))))
        print("RMSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))))
        print("MAE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))))
        print("R2 model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(r2_score(y_test, y_pred_rfg), 2))))
        print(" ")
        print("==================================================")

MSE model Linear Regression data Train dengan 100 fitur: 38.83
RMSE model Linear Regression data Train dengan 100 fitur:6.23
MAE model Linear Regression data Train dengan 100 fitur:4.77
R2 model Linear Regression data Train dengan 100 fitur:0.8
----------------------------
MSE model Linear Regression data Test dengan 100 fitur:177.08
RMSE model Linear Regression data Test dengan 100 fitur:13.31
MAE model Linear Regression data Test dengan 100 fitur:10.57
R2 model Linear Regression data Test dengan 100 fitur:0.03
 
CPU times: user 5.05 s, sys: 18.6 ms, total: 5.07 s
Wall time: 5.07 s


## 200 Trees

In [11]:
%%time

#tentukan metode scoring yang digunakan
metrics = {'rmse': 'neg_root_mean_squared_error',
               'mse': 'neg_mean_squared_error',
               'mae': 'neg_mean_absolute_error',
               'r2': 'r2'} 

n_feat = [100]
n_trees = [200]

for nfeat in n_feat:
    for ntrees in n_trees: 
        print("==================================================")
        
        #ambil n fitur input hasil seleksi fitur SFS
        x_train_selected = x_train[feat_names].iloc[:,0:nfeat]
        x_test_selected = x_test[feat_names].iloc[:,0:nfeat]

        #Create a Random Forest Regression
        rfg_model = RandomForestRegressor(n_estimators=ntrees, random_state=100)
        
        #Train the model using the training sets
        rfg_model.fit(x_train_selected, y_train)
        y_pred_rfg = rfg_model.predict(x_test_selected)

        #hitung score model dari data train
        scores = cross_validate(rfg_model, x_train_selected, y_train, scoring=metrics, cv=cv, return_train_score=True)

        print("MSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur: " 
              + str(abs(round(scores['train_mse'].mean(), 2))))
        print("RMSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs(round(scores['train_rmse'].mean(), 2))))
        print("MAE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs(round(scores['train_mae'].mean(), 2))))
        print("R2 model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs((round(scores['train_r2'].mean(), 2)))))    
        print("----------------------------")
        print("MSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_rfg), 2))))
        print("RMSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))))
        print("MAE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))))
        print("R2 model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(r2_score(y_test, y_pred_rfg), 2))))
        print(" ")
        print("==================================================")

MSE model Linear Regression data Train dengan 100 fitur: 38.78
RMSE model Linear Regression data Train dengan 100 fitur:6.22
MAE model Linear Regression data Train dengan 100 fitur:4.77
R2 model Linear Regression data Train dengan 100 fitur:0.8
----------------------------
MSE model Linear Regression data Test dengan 100 fitur:175.34
RMSE model Linear Regression data Test dengan 100 fitur:13.24
MAE model Linear Regression data Test dengan 100 fitur:10.57
R2 model Linear Regression data Test dengan 100 fitur:0.02
 
CPU times: user 6.7 s, sys: 13.7 ms, total: 6.71 s
Wall time: 6.7 s
